# Evo-Bayes-Agent

I use `bayesian-optimization==0.6`, my backend pretty much stick with this version, so migrating will break the code.

In [ ]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import seaborn as sns
import random
from bayes_opt import BayesianOptimization
sns.set()

In [ ]:
def get_state(data, t, n):
    d = t - n + 1
    block = data[d : t + 1] if d >= 0 else -d * [data[0]] + data[0 : t + 1]
    res = []
    for i in range(n - 1):
        res.append(block[i + 1] - block[i])
    return np.array([res])

TSLA Time Period: **Mar 23, 2018 - Mar 23, 2019**

In [ ]:
from Ionomy import IonPanda, BitTA
from decouple import config

MARKET = 'btc-hive'
CURRENCY = 'hive'
BASE = 'btc'
TIME = 'day'

bta = BitTA(config('TREX_KEY'), config('TREX_SECRET'))
ionpd = IonPanda(config('ION_KEY'), config('ION_SECRET'))
bta.update(CURRENCY, BASE, TIME)

df = bta.df

In [ ]:
close = df.close.values.tolist()
window_size = 30
skip = 5
l = len(close) - 1

In [ ]:
accbest = 0.0
NN_BAYESIAN = BayesianOptimization(
    find_best_agent,
    {
        'window_size': (2, 50),
        'skip': (1, 15),
        'population_size': (1, 50),
        'sigma': (0.01, 0.99),
        'learning_rate': (0.000001, 0.49),
        'size_network': (10, 1000),
    },
)
NN_BAYESIAN.maximize(init_points = 30, n_iter = 50, acq = 'ei', xi = 0.0)

In [ ]:
print('Best AGENT accuracy value: %f' % NN_BAYESIAN.res['max']['max_val'])
print('Best AGENT parameters: ', NN_BAYESIAN.res['max']['max_params'])

In [ ]:
df = pd.DataFrame.from_records([{"target": item["target"], **item["params"]} for item in NN_BAYESIAN.res])

In [ ]:
best_params = df.iloc[df['target'].idxmax()].to_dict()
best_params

#### My selected parameters

In [ ]:
best_agent(
    window_size = 30, 
    skip = 1, 
    population_size = 15, 
    sigma = 0.1, 
    learning_rate = 0.03, 
    size_network = 500
)

#### bayesian parameters

In [ ]:
best_agent(
    window_size = int(np.around(best_params['window_size'])), 
    skip = int(np.around(best_params['skip'])), 
    population_size = int(np.around(best_params['population_size'])), 
    sigma = best_params['sigma'], 
    learning_rate = best_params['learning_rate'], 
    size_network = int(np.around(best_params['size_network']))
)

#### My selected parameters

In [ ]:
# model = Model(input_size = 30, 
#               layer_size = 500, 
#               output_size = 3)
# agent = Agent(population_size = 15, 
#               sigma = 0.1, 
#               learning_rate = 0.03, 
#               model = model, 
#               money = 10000, 
#               max_buy = 5, 
#               max_sell = 5, 
#               skip = 1, 
#               window_size = 30)
# agent.fit(500, 100)
# agent.buy()

#### bayesian parameters

In [ ]:
model = Model(input_size = int(np.around(best_params['window_size'])), 
              layer_size = int(np.around(best_params['size_network'])), 
              output_size = 3)
agent = Agent(population_size = int(np.around(best_params['population_size'])), 
              sigma = best_params['sigma'], 
              learning_rate = best_params['learning_rate'], 
              model = model, 
              money = 10000, 
              max_buy = 5, 
              max_sell = 5, 
              skip = int(np.around(best_params['skip'])), 
              window_size = int(np.around(best_params['window_size'])))
agent.fit(500, 100)
agent.buy()